In [ ]:
import pandas as pd
import pyodbc 

In [ ]:
df = pd.read_csv('Repo_in2023.csv')

In [ ]:
df.columns

In [ ]:
df['created_at'] = pd.to_datetime(df['created_at'])
# Extract the date part from the datetime and convert back to string format :
df['created_at'] = df['created_at'].dt.strftime('%Y-%m-%d')

In [ ]:
df['contributors']  = df['contributors'].apply(eval)
df['contributions_count']  = df['contributions_count'].apply(eval)

#### Create Conection avec la base de donnee :

In [ ]:
# Créer un connexion dans le database :

driver = 'SQL Server'
server = 'LAPTOP-6D8J0VI4\SQLEXPRESS'
database = 'github_DB'

conn = pyodbc.connect(f'''
                        DRIVER={driver};
                        SERVER={server};
                        DATABASE={database};
                        Trusted_Connection=yes
                       ''')
cursor = conn.cursor()
conn

In [ ]:
cursor.execute('''
               
CREATE TABLE [Languages] (
  [Id] INT IDENTITY(0,1),
  [name] VARCHAR(255),
  PRIMARY KEY ([Id])
);

CREATE TABLE [Owner] (
  [Id] INT IDENTITY(0,1),
  [name] VARCHAR(255),
  PRIMARY KEY ([Id])
);

CREATE TABLE [Repositorys] (
  [Id] INT IDENTITY(0,1),
  [name] VARCHAR(255),
  [Url] VARCHAR(255),
  [description] NVARCHAR(max),
  [forks] INT,
  [stars] INT,
  [created_at] DATE,
  [open_issues] INT,
  [contributors_count] INT,
  [language_id] INT,
  [owner_id] INT,
  PRIMARY KEY ([Id]),
  CONSTRAINT [FK_Repositorys_language_id]
    FOREIGN KEY ([language_id])
    REFERENCES [Languages]([Id]),
  CONSTRAINT [FK_Repositorys_owner_id]
    FOREIGN KEY ([owner_id])
    REFERENCES [Owner]([Id])
);


CREATE TABLE [Contributors] (
  [Id] INT IDENTITY(0,1),
  [name] VARCHAR(255),
  [number_commit] INT
  PRIMARY KEY ([Id])
);

CREATE TABLE [Contributor_bridge] (
  [Id_repo] INT,
  [Id_contributor] INT
);

-- Add Constraints

ALTER TABLE [Contributor_bridge]
ADD CONSTRAINT [FK_Contributor_bridge_Id_repo]
  FOREIGN KEY ([Id_repo])
  REFERENCES [Repositorys]([Id]);

ALTER TABLE [Contributor_bridge]
ADD CONSTRAINT [FK_Contributor_bridge_Id_contributor]
  FOREIGN KEY ([Id_contributor])
  REFERENCES [Contributors]([Id]);
        ''')

cursor.commit()

In [ ]:
tables = cursor.tables(tableType='TABLE')

for table_info in tables:
    table_name = table_info.table_name
    print(table_name)

#### Insert Into language Table :

In [ ]:
uique_Language  = list(df['language'].unique())

for languge in uique_Language :
    cursor.execute('INSERT INTO Languages(name) VALUES (?)',languge)

cursor.commit()

In [ ]:
len(uique_Language)

#### Insert Into owner Table :

In [ ]:
owners  = list(df['owner'].unique())

for owner in owners :
    cursor.execute('INSERT INTO Owner(name) VALUES (?)',owner)

cursor.commit()

#### Insert Into Contributor Table :

In [ ]:
list_contributions_count = list(df['contributions_count'])
list_contributions = list(df['contributors'])

list_contributions = [element for sublist in list_contributions for element in sublist]
list_contributions_count = [element for sublist in list_contributions_count for element in sublist]

# Create a dictionary from the lists :
data = {'Contributor': list_contributions, 'Contribution_Count': list_contributions_count}

# Create a DataFrame from the dictionary :
Conterbuter_df = pd.DataFrame(data)
Conterbuter_df = Conterbuter_df.drop_duplicates()
Conterbuter_df.duplicated().sum()


In [ ]:
Conterbuter_df

In [ ]:
for index, row in Conterbuter_df.iterrows() :
    cursor.execute('INSERT INTO Contributors(name,number_commit) VALUES (?,?)',row['Contributor'],row['Contribution_Count'])

cursor.commit()

#### Insert Into Repos Table :

In [ ]:
for index, row in df.iterrows() :

    cursor.execute(f"SELECT id FROM Owner WHERE name = '{row['owner']}'")
    owner_id = cursor.fetchone()[0]

    cursor.execute(f"SELECT id FROM Languages WHERE name = '{row['language']}'")
    language_id = cursor.fetchone()[0]
    
    cursor.execute('''INSERT INTO Repositorys(name,Url,description,forks,stars,created_at,open_issues,contributors_count,language_id,owner_id)
                    VALUES (?,?,?,?,?,?,?,?,?,?)''',
    row['name'],
    row['url'],
    str(row['description']),
    row['forks'],
    row['stars'],
    row['created_at'],
    row['open_issues'],
    row['contributors_count'],
    language_id, 
    owner_id
    )

cursor.commit()

#### insert into bridg :

In [ ]:
df.head(1)

In [ ]:
Conterbuter_df_list = list(Conterbuter_df['Contributor'])

for index, row in df.iterrows() :

    for contr in row['contributors'] :
        Contributors_id = Conterbuter_df_list.index(contr)
        print(index,'=> ',Contributors_id)
        cursor.execute('INSERT INTO Contributor_bridge(Id_repo,Id_contributor) VALUES (?,?)',index,Contributors_id)
cursor.commit()

In [ ]:
conn.close()